In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file_path = ('/content/final_data.csv')
df = pd.read_csv(file_path)
df.head()



DATA CLEANING

In [ ]:
# Check for missing values and fill them
df.isnull().sum()
df = df.dropna()
print(df.isnull().sum())
print(df.info())
print(df.head())

# Convert relevant columns to numeric
df['QUANTITYORDERED'] = pd.to_numeric(df['QUANTITYORDERED'], errors='coerce')
df['PRICEEACH'] = pd.to_numeric(df['PRICEEACH'], errors='coerce')
print(df.info())




DATA ANALYSIS

In [ ]:
# Standardize date format
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])
print(df.head())


# Combine product Categories
df['PRODUCTLINE'] = df['PRODUCTLINE'].str.lower().str.strip()
print(df['PRODUCTLINE'].unique())


# total spending calculation and defining customer segments
customer_segments = df.groupby('CUSTOMERNAME').agg({
    'QUANTITYORDERED': 'sum',
    'PRICEEACH': 'sum'
}).reset_index()

customer_segments['TOTAL_SPENDING'] = customer_segments['QUANTITYORDERED'] * customer_segments['PRICEEACH']
customer_segments['SEGMENT'] = pd.cut(customer_segments['TOTAL_SPENDING'], bins=3, labels=['Low-Value', 'Mid-Value', 'High-Value'])
print(customer_segments)


# monthly trends analysis
df['MONTH'] = df['ORDERDATE'].dt.to_period('M')
monthly_trends = df.groupby('MONTH')['QUANTITYORDERED'].sum().reset_index()
monthly_trends['MONTH'] = monthly_trends['MONTH'].astype(str)
print(monthly_trends)


In [ ]:
# Customer segmentation based on purchase frequency and total spending
customer_data = df.groupby('CUSTOMERNAME').agg({
    'ORDERNUMBER': 'count',
    'QUANTITYORDERED': 'sum'
})
customer_data.columns = ['PurchaseFrequency', 'TotalSpending']
print(customer_data.head())


In [ ]:
#Customer Lifetime Value (CLTV) Calculation
#cltv = total_revenue / num_customers

customer_revenue = df.groupby('CUSTOMERNAME')['SALES'].sum().reset_index()

total_revenue = customer_revenue['SALES'].sum()
num_customers = customer_revenue['CUSTOMERNAME'].nunique()
cltv = total_revenue / num_customers

print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Number of Customers: {num_customers}")
print(f"Customer Lifetime Value (CLTV): ${cltv:,.2f}")

VISUALIZATION

In [ ]:
# Monthly Trends (by quantity)
plt.figure(figsize=(14, 7))
sns.lineplot(data=monthly_trends, x='MONTH', y='QUANTITYORDERED')
plt.title('Monthly Trends of Quantity Ordered')
plt.xlabel('Month')
plt.ylabel('Quantity Ordered')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Monthly Sales
plt.figure(figsize=(14, 7))
sns.barplot(df, x='MONTH', y='SALES')
plt.title('Monthly Sales')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Customer Segmentation Pie Chart
plt.figure(figsize=(10, 6))
df['PRODUCTLINE'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('PRODUCT CATEGORY')
plt.axis('equal')
plt.show()


In [ ]:
# Total Revenue by Customer
plt.figure(figsize=(10, 6))
sns.barplot(x='SALES', y='CUSTOMERNAME', data=customer_revenue.sort_values('SALES', ascending=False).head(10))
plt.title('Top 10 Customers by Total Revenue')
plt.xlabel('Total Revenue')
plt.ylabel('Customer Name')
plt.show()

In [ ]:
# Monthly Sales Trend
df['Month'] = df['ORDERDATE'].dt.to_period('M')
monthly_sales = df.groupby('Month')['SALES'].sum().reset_index()
monthly_sales['Month'] = monthly_sales['Month'].astype(str)

plt.figure(figsize=(14, 7))
sns.lineplot(x='Month', y='SALES', data=monthly_sales)
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Sales Distribution by Product Line
plt.figure(figsize=(14, 7))
sns.barplot(x='COUNTRY', y='SALES', data=df)
plt.title('Sales Distribution by Country')
plt.xlabel('Country')
plt.ylabel('Sales')
plt.xticks(rotation=45)
plt.show()
